In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
from numba import njit

In [7]:
fpath = "/Users/saadjansari/Documents/Projects/AMSOS/resultsSummit/checks/eq"
simnames = [
    "f1",
    "f1_5",
    "f2",
    "f2_5",
    "f5",
    "f10",
]
simlabels = [
    "Filamin = 1.0",
    "Filamin = 1.5",
    "Filamin = 2.0",
    "Filamin = 2.5",
    "Filamin = 5.0",
    "Filamin = 10.0",
]
fconc = [1,1.5,2,2.5,5,10]
simpaths_actin = [os.path.join(fpath, ii, "result/result0-399/SylinderAscii_6.dat") for ii in simnames]
simpaths_xlinks = [os.path.join(fpath, ii, "result/result0-399/ProteinAscii_6.dat") for ii in simnames]

In [3]:
# Filament class
class Filament():
    def __init__(self, pos0, pos1, radius,gid):
        self.radius = radius
        self.pos0 = pos0
        self.pos1 = pos1
        self.gid = gid
    def GetCenter(self,boxsize):
        return getMean(self.pos0, self.pos1,boxsize)
    def GetLength(self,boxsize):
        xi = getDistance(self.pos1,self.pos0,boxsize)
        return np.linalg.norm( xi)
    def GetOrientation(self):
        xi = self.pos1 - self.pos0
        return xi/np.sqrt(xi.dot(xi))

    def Plot3(self,ax, col="red"):
        ax.plot3D( [self.pos0[0], self.pos1[0]], [self.pos0[1], self.pos1[1]], [self.pos0[2], self.pos1[2]], col)
    def GetStringtoWrite(self):
        return 'C {0} {1} {2:0.6f} {3:0.6f} {4:0.6f} {5:0.6f} {6:0.6f} {7:0.6f}\n'.format(
        self.gid, self.radius,
        self.pos0[0], self.pos0[1], self.pos0[2],
        self.pos1[0], self.pos1[1], self.pos1[2])
    def __repr__(self):
        return "Filament()"
    def __str__(self):
        return 'Filament {0}:\n  pos0: {1}\n  pos1: {2}\n  radius: {3}'.format(self.gid, self.pos0, self.pos1,self.radius)

# Protein class
class Protein():
    def __init__(self, pos0, pos1, link0, link1, gid):
        self.pos0 = pos0
        self.pos1 = pos1
        self.link0 = link0
        self.link1 = link1
        self.gid = gid
    def GetCenter(self,boxsize):
        return getMean(self.pos0, self.pos1,boxsize)
    def GetLength(self,boxsize):
        xi = getDistance(self.pos1,self.pos0,boxsize)
        return np.linalg.norm( xi)
    def GetOrientation(self):
        if link0 != -1 and link1 != -1:
            xi = self.pos1 - self.pos0
            return xi/np.sqrt( xi.dot(xi))
        else:
            return None

    def Plot3(self,ax,col="blue"):
        ax.plot3D( [self.pos0[0], self.pos1[0]], [self.pos0[1], self.pos1[1]], [self.pos0[2], self.pos1[2]], col)
    def GetStringtoWrite(self):
        return 'P {0} 0 {2:0.6f} {3:0.6f} {4:0.6f} {5:0.6f} {6:0.6f} {7:0.6f} {8} {9} \n'.format(
        self.gid, self.radius,
        self.pos0[0], self.pos0[1], self.pos0[2],
        self.pos1[0], self.pos1[1], self.pos1[2],
        self.link0, self.link1)
    def __repr__(self):
        return "Protein()"
    def __str__(self):
        return 'Protein {0}:\n  pos0: {1}\n  pos1: {2}\n  Links: {3}--{4}'.format(self.gid, self.pos0, self.pos1, self.link0, self.link1)

    
def load_filaments( fpath):
    
    flist = []
    with open(fpath, 'r') as file1:
        filecontent = file1.readlines()
        for line in filecontent:
            if line.startswith('C'):
                data = line.split()
                gid = int(data[1])
                radius = float(data[2])
                pos0 = np.array([float(data[3]), float(data[4]), float(data[5])])
                pos1 = np.array([float(data[6]), float(data[7]), float(data[8])])
                flist.append( Filament(pos0, pos1, radius,gid))
    return flist

def load_proteins( fpath):
    
    plist = []
    with open(fpath, 'r') as file2:
        filecontent = file2.readlines()
        for line in filecontent:
            if line.startswith('P'):
                data = line.split()
                gid = int(data[1])
                pos0 = np.array([float(data[3]), float(data[4]), float(data[5])])
                pos1 = np.array([float(data[6]), float(data[7]), float(data[8])])
                link0 = int(data[9])
                link1 = int(data[10])
                plist.append( Protein(pos0, pos1, link0, link1, gid))
    return plist

In [4]:
def getCC( ftlist, ptlist):
    
    cct2 = np.zeros(nF,dtype=bool)
    # Create a graph for filaments
    g = nx.Graph()
    g.add_nodes_from( np.arange( len(ftlist)).tolist() )

    # add edges to the graph (each edge represents a binding xlink)
    for p in ptlist:
        if p.link0 != -1 and p.link1 != -1:
            g.add_edge(p.link0, p.link1)

    # find connected component largest
    cc = list( max(nx.connected_components(g), key=len) )
    #cct2[cc] = True
    #cct2[[it for i in range(len(cc))], cc] = True

    return cc,cct2

In [5]:
def calc_nematic_order(orient_array):
    # Calculates the maximum eigenvalue of the nematic tensor Q

    # calculate Q tensor
    Q = calc_nematic_tensor(orient_array)
    S = np.sqrt(np.tensordot(Q, Q)*1.5)
    return S

@njit
def calc_polar_order(orient_array):
    # Calculates the polar order given the array of filament orientations

    # Initialize P vector
    Pvec = np.zeros(3)

    # Take a mean of all orientations
    for irow in orient_array:
        Pvec += orient_array[irow,:]
    Pvec = Pvec / orient_array.shape[0]
    P = np.linalg.norm( Pvec)
    return P


@njit
def calc_nematic_tensor( orient_array):
    # Calculates the nematic tensor Q

    # initialize Q tensor
    Q = np.zeros((3,3))
    # sum over all filaments, taking their outer products
    for irow in np.arange(orient_array.shape[0]):
        Q += np.outer( orient_array[irow,:], orient_array[irow,:])

    # complete mean calculation by dividing by number of filaments and subtracting identity.
    Q = Q/orient_array.shape[0] - np.identity(3)/3
    return Q

In [21]:
# Find clusters
for idx in range(len(simpaths_actin)):
    flist = load_filaments( simpaths_actin[idx])
    plist = load_proteins(simpaths_xlinks[idx])
    nP = len(plist)
    nF = len(flist)

    cc,cct2 = getCC(flist,plist)
    
    # get filaments in cluster
    flist_trimmed = []
    idx = np.where(cct2)[0].tolist()
    for el in idx:
        flist_trimmed.append( flist[el])
    
    # Bulk Nematic order
    orientList = np.zeros( (len(flist),3))
    for ifil in range(len(flist)):
        orientList[ifil,:] = flist[ifil].GetOrientation()
    print(calc_nematic_order(orientList))
    
    # Cluster Nematic order
    orientList = np.zeros( (len(flist_trimmed),3))
    for ifil in range(len(flist_trimmed)):
        orientList[ifil,:] = flist_trimmed[ifil].GetOrientation()
    print(calc_nematic_order(orientList))

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
